# 🗃️ SQL Helpers

> Comprehensive SQL helper library for database operations with multi-database support.

In [ ]:
#| default_exp utils_sql

In [ ]:
#| export

from fastsql import *
from sqlalchemy import text
import os
import re
import logging
from typing import List, Dict, Any, Optional
from contextlib import contextmanager
from nbdev.showdoc import show_doc

# Module-level logger - configured by app via configure_logging()
logger = logging.getLogger(__name__)

## 🎯 Overview

This module provides a **batteries-included SQL toolkit** for building database-heavy applications:

| Category | Functions | Purpose |
|----------|-----------|--------|
| 🔍 Query Registry | `run_id`, `validate_params` | Execute queries by ID from centralized registries |
| ➕ Insert-Only | `insert_only`, `bulk_insert_only` | Insert new records, skip conflicts |
| 🔄 Upsert | `upsert`, `bulk_upsert` | Insert or update existing records |
| 📝 CRUD | `get_by_id`, `update_record`, `delete_record`, `bulk_delete` | Standard database operations |
| 🔧 Utilities | `with_transaction`, `paginate_sql`, `batch_execute` | Transaction management & helpers |
| 💰 Money | `to_cents`, `from_cents` | Currency conversion for `int`-only storage |

---

## 🏗️ Architecture

```
┌─────────────────────────────────────────────────────────────────┐
│                     SQL Utilities Layer                         │
├─────────────────────────────────────────────────────────────────┤
│  🔍 Query Registry        │  Execute queries by ID              │
│  ➕ Insert Operations     │  insert_only, bulk_insert_only      │
│  🔄 Upsert Operations     │  upsert, bulk_upsert                │
│  📝 CRUD Operations       │  get, update, delete (single/bulk)  │
│  🔧 Utilities             │  transactions, pagination, batching │
│  💰 Money Helpers         │  cents ↔ dollars conversion         │
├─────────────────────────────────────────────────────────────────┤
│  DB_TYPE Detection: PostgreSQL / SQLite (auto-switch syntax)    │
└─────────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────────┐
│              fastsql Database Connection                        │
└─────────────────────────────────────────────────────────────────┘
```

---

## 📚 Quick Reference

### Conflict Handling

| Operation | On Conflict | Use Case |
|-----------|-------------|----------|
| `insert_only` | **Skip** (DO NOTHING) | Append-only logs, idempotent imports |
| `upsert` | **Update** existing | Sync external data, refresh caches |

### Performance Tips

| Scenario | Recommended | Why |
|----------|-------------|-----|
| Single record | `insert_only`, `upsert` | Simple, immediate |
| 10+ records | `bulk_*` variants | 10-100x faster |
| 100+ records | `batch_execute` | Commits per batch, memory-safe |
| Multi-table changes | `with_transaction` | All-or-nothing commits |

---

## 🔍 Database Type Detection

Automatically detects PostgreSQL vs SQLite from `DB_TYPE` environment variable to generate appropriate SQL syntax.

In [ ]:
#| export

def get_db_type():
    """Get database type from environment variable"""
    return os.getenv("DB_TYPE", "SQLITE").upper()

In [ ]:
show_doc(get_db_type)

---

### get_db_type

>      get_db_type ()

*Get database type from environment variable*

---

## 🎯 Query Registry

Execute queries by ID from a centralized registry with automatic parameter validation.

| Function | Purpose |
|----------|--------|
| `run_id` | Execute a registered query by its ID |
| `validate_params` | Ensure all `:param` placeholders have values |

In [ ]:
#| export

def _extract_params(sql: str) -> List[str]:
    """Extract parameter names from SQL string (e.g., :param_name)"""
    return re.findall(r':(\w+)', sql)

def validate_params(sql: str, params: Dict[str, Any]) -> None:
    """Validate that all required parameters are provided"""
    required = _extract_params(sql)
    provided = set(params.keys()) if params else set()
    missing = set(required) - provided
    
    if missing:
        raise ValueError(f"Missing required parameters: {', '.join(missing)}")

def run_id(db: Database, registry: Dict[str, str], query_id: str, params: Optional[Dict[str, Any]] = None) -> Any:
    """Execute a query by ID from a query registry."""
    # Get query from registry
    if query_id not in registry:
        raise ValueError(f"Query ID '{query_id}' not found in registry")
    
    sql = registry[query_id]
    params = params or {}
    
    # Validate parameters
    validate_params(sql, params)
    
    # Execute query
    try:
        result = db.conn.execute(text(sql), params)
        return result
    except Exception as e:
        logger.error(f"Query '{query_id}' failed: {e}", exc_info=True)
        raise Exception(f"Failed to execute query '{query_id}': {str(e)}") from e


In [ ]:
show_doc(run_id)

---

### run_id

>      run_id (db:fastsql.core.Database, registry:Dict[str,str], query_id:str,
>              params:Optional[Dict[str,Any]]=None)

*Execute a query by ID from a query registry.*

In [ ]:
show_doc(validate_params)

---

### validate_params

>      validate_params (sql:str, params:Dict[str,Any])

*Validate that all required parameters are provided*

---

## ➕ Insert-Only

Insert new records **only if they don't exist** — conflicts are silently skipped.

| Function | Records | SQL Generated |
|----------|---------|---------------|
| `insert_only` | Single | `ON CONFLICT DO NOTHING` (PG) / `INSERT OR IGNORE` (SQLite) |
| `bulk_insert_only` | Multiple | Same, with batch execution |

> 💡 **Use case**: Idempotent imports, append-only audit logs, webhook deduplication

In [ ]:
#| export

def insert_only(db: Database, table_name: str, record: Dict[str, Any], conflict_cols: List[str], auto_commit: bool = True) -> None:
    """Insert a single record only if it doesn't exist (ignores conflicts)."""
    db_type = get_db_type()
    columns = list(record.keys())
    placeholders = [f":{col}" for col in columns]
    
    if db_type == "POSTGRESQL":
        # ON CONFLICT DO NOTHING
        conflict_clause = f"ON CONFLICT ({', '.join(conflict_cols)}) DO NOTHING"
        sql = f"""
            INSERT INTO {table_name} ({', '.join(columns)})
            VALUES ({', '.join(placeholders)})
            {conflict_clause}
        """
    else:  # SQLite
        # INSERT OR IGNORE
        sql = f"""
            INSERT OR IGNORE INTO {table_name} ({', '.join(columns)})
            VALUES ({', '.join(placeholders)})
        """
    
    try:
        db.conn.execute(text(sql), record)
        if auto_commit:
            db.conn.commit()
    except Exception as e:
        logger.error(f"Insert into '{table_name}' failed: {e}", exc_info=True)
        if auto_commit:
            db.conn.rollback()
        raise Exception(f"Failed to insert record into {table_name}: {str(e)}") from e

def bulk_insert_only(db: Database, table_name: str, records: List[Dict[str, Any]], conflict_cols: List[str], auto_commit: bool = True) -> None:
    """Insert multiple records, skipping conflicts (optimized batch operation)."""
    if not records:
        return
    
    db_type = get_db_type()
    columns = list(records[0].keys())
    placeholders = [f":{col}" for col in columns]
    
    if db_type == "POSTGRESQL":
        conflict_clause = f"ON CONFLICT ({', '.join(conflict_cols)}) DO NOTHING"
        sql = f"""
            INSERT INTO {table_name} ({', '.join(columns)})
            VALUES ({', '.join(placeholders)})
            {conflict_clause}
        """
    else:  # SQLite
        sql = f"""
            INSERT OR IGNORE INTO {table_name} ({', '.join(columns)})
            VALUES ({', '.join(placeholders)})
        """
    
    try:
        for record in records:
            db.conn.execute(text(sql), record)
        if auto_commit:
            db.conn.commit()
    except Exception as e:
        logger.error(f"Bulk insert into '{table_name}' failed: {e}", exc_info=True)
        if auto_commit:
            db.conn.rollback()
        raise Exception(f"Failed to bulk insert into {table_name}: {str(e)}") from e

In [ ]:
show_doc(bulk_insert_only)

---

### bulk_insert_only

>      bulk_insert_only (db:fastsql.core.Database, table_name:str,
>                        records:List[Dict[str,Any]], conflict_cols:List[str],
>                        auto_commit:bool=True)

*Insert multiple records, skipping conflicts (optimized batch operation).*

In [ ]:
show_doc(insert_only)

---

### insert_only

>      insert_only (db:fastsql.core.Database, table_name:str,
>                   record:Dict[str,Any], conflict_cols:List[str],
>                   auto_commit:bool=True)

*Insert a single record only if it doesn't exist (ignores conflicts).*

---

## 🔄 Upsert

Insert new records **or update existing ones** on conflict.

| Function | Records | SQL Generated |
|----------|---------|---------------|
| `upsert` | Single | `ON CONFLICT DO UPDATE` (PG) / `INSERT OR REPLACE` (SQLite) |
| `bulk_upsert` | Multiple | Same, with batch execution |

> 💡 **Use case**: Syncing external API data, refreshing caches, user settings

In [ ]:
#| export

def upsert(db: Database, table_name: str, record: Dict[str, Any], 
           conflict_cols: List[str], update_cols: Optional[List[str]] = None, auto_commit: bool = True) -> None:
    """Insert a record or update if it exists (upsert)."""
    db_type = get_db_type()
    columns = list(record.keys())
    placeholders = [f":{col}" for col in columns]
    
    # Determine which columns to update
    if update_cols is None:
        update_cols = [col for col in columns if col not in conflict_cols]
    
    if db_type == "POSTGRESQL":
        # ON CONFLICT DO UPDATE
        update_set = ', '.join([f"{col} = EXCLUDED.{col}" for col in update_cols])
        sql = f"""
            INSERT INTO {table_name} ({', '.join(columns)})
            VALUES ({', '.join(placeholders)})
            ON CONFLICT ({', '.join(conflict_cols)}) 
            DO UPDATE SET {update_set}
        """
    else:  # SQLite
        # INSERT OR REPLACE
        sql = f"""
            INSERT OR REPLACE INTO {table_name} ({', '.join(columns)})
            VALUES ({', '.join(placeholders)})
        """
    
    try:
        db.conn.execute(text(sql), record)
        if auto_commit:
            db.conn.commit()
    except Exception as e:
        logger.error(f"Upsert into '{table_name}' failed: {e}", exc_info=True)
        if auto_commit:
            db.conn.rollback()
        raise Exception(f"Failed to upsert record into {table_name}: {str(e)}") from e

def bulk_upsert(db: Database, table_name: str, records: List[Dict[str, Any]], 
                conflict_cols: List[str], update_cols: Optional[List[str]] = None, auto_commit: bool = True) -> None:
    """Insert or update multiple records (optimized batch operation)."""
    if not records:
        return
    
    db_type = get_db_type()
    columns = list(records[0].keys())
    placeholders = [f":{col}" for col in columns]
    
    # Determine which columns to update
    if update_cols is None:
        update_cols = [col for col in columns if col not in conflict_cols]
    
    if db_type == "POSTGRESQL":
        update_set = ', '.join([f"{col} = EXCLUDED.{col}" for col in update_cols])
        sql = f"""
            INSERT INTO {table_name} ({', '.join(columns)})
            VALUES ({', '.join(placeholders)})
            ON CONFLICT ({', '.join(conflict_cols)}) 
            DO UPDATE SET {update_set}
        """
    else:  # SQLite
        sql = f"""
            INSERT OR REPLACE INTO {table_name} ({', '.join(columns)})
            VALUES ({', '.join(placeholders)})
        """
    
    try:
        for record in records:
            db.conn.execute(text(sql), record)
        if auto_commit:
            db.conn.commit()
    except Exception as e:
        logger.error(f"Bulk upsert into '{table_name}' failed: {e}", exc_info=True)
        if auto_commit:
            db.conn.rollback()
        raise Exception(f"Failed to bulk upsert into {table_name}: {str(e)}") from e

In [ ]:
show_doc(bulk_upsert)

---

### bulk_upsert

>      bulk_upsert (db:fastsql.core.Database, table_name:str,
>                   records:List[Dict[str,Any]], conflict_cols:List[str],
>                   update_cols:Optional[List[str]]=None, auto_commit:bool=True)

*Insert or update multiple records (optimized batch operation).*

In [ ]:
show_doc(upsert)

---

### upsert

>      upsert (db:fastsql.core.Database, table_name:str, record:Dict[str,Any],
>              conflict_cols:List[str], update_cols:Optional[List[str]]=None,
>              auto_commit:bool=True)

*Insert a record or update if it exists (upsert).*

---

## 📝 CRUD

Standard Create, Read, Update, Delete operations.

| Function | Operation | Description |
|----------|-----------|-------------|
| `get_by_id` | **Read** | Fetch single record by primary key |
| `update_record` | **Update** | Modify record fields by ID |
| `delete_record` | **Delete** | Remove single record by ID |
| `bulk_delete` | **Delete** | Remove multiple records by ID list |

In [ ]:
#| export

def get_by_id(db: Database, table_name: str, id_value: Any, id_col: str = "id") -> Any:
    """Get a single record by ID."""
    sql = f"SELECT * FROM {table_name} WHERE {id_col} = :id_value"
    try:
        result = db.conn.execute(text(sql), {"id_value": id_value})
        return result
    except Exception as e:
        logger.error(f"Get by ID from '{table_name}' failed: {e}", exc_info=True)
        raise Exception(f"Failed to get record from {table_name}: {str(e)}") from e

def update_record(db: Database, table_name: str, id_value: Any, 
                  id_col: str = "id", auto_commit: bool = True, **updates) -> None:
    """Update a single record by ID."""
    if not updates:
        return
    
    set_clause = ', '.join([f"{col} = :{col}" for col in updates.keys()])
    sql = f"UPDATE {table_name} SET {set_clause} WHERE {id_col} = :id_value"
    
    params = {**updates, "id_value": id_value}
    
    try:
        db.conn.execute(text(sql), params)
        if auto_commit:
            db.conn.commit()
    except Exception as e:
        logger.error(f"Update record in '{table_name}' failed: {e}", exc_info=True)
        if auto_commit:
            db.conn.rollback()
        raise Exception(f"Failed to update record in {table_name}: {str(e)}") from e

def delete_record(db: Database, table_name: str, id_value: Any, id_col: str = "id", auto_commit: bool = True) -> None:
    """Delete a single record by ID."""
    sql = f"DELETE FROM {table_name} WHERE {id_col} = :id_value"
    
    try:
        db.conn.execute(text(sql), {"id_value": id_value})
        if auto_commit:
            db.conn.commit()
    except Exception as e:
        logger.error(f"Delete from '{table_name}' failed: {e}", exc_info=True)
        if auto_commit:
            db.conn.rollback()
        raise Exception(f"Failed to delete record from {table_name}: {str(e)}") from e

def bulk_delete(db: Database, table_name: str, id_list: List[Any], id_col: str = "id", auto_commit: bool = True) -> None:
    """Delete multiple records by ID list."""
    if not id_list:
        return
    
    # Create placeholder for IN clause
    placeholders = ', '.join([f":id_{i}" for i in range(len(id_list))])
    sql = f"DELETE FROM {table_name} WHERE {id_col} IN ({placeholders})"
    
    # Build params dict
    params = {f"id_{i}": val for i, val in enumerate(id_list)}
    
    try:
        db.conn.execute(text(sql), params)
        if auto_commit:
            db.conn.commit()
    except Exception as e:
        logger.error(f"Bulk delete from '{table_name}' failed: {e}", exc_info=True)
        if auto_commit:
            db.conn.rollback()
        raise Exception(f"Failed to bulk delete from {table_name}: {str(e)}") from e

In [ ]:
show_doc(bulk_delete)

---

### bulk_delete

>      bulk_delete (db:fastsql.core.Database, table_name:str, id_list:List[Any],
>                   id_col:str='id', auto_commit:bool=True)

*Delete multiple records by ID list.*

In [ ]:
show_doc(delete_record)

---

### delete_record

>      delete_record (db:fastsql.core.Database, table_name:str, id_value:Any,
>                     id_col:str='id', auto_commit:bool=True)

*Delete a single record by ID.*

In [ ]:
show_doc(update_record)

---

### update_record

>      update_record (db:fastsql.core.Database, table_name:str, id_value:Any,
>                     id_col:str='id', auto_commit:bool=True, **updates)

*Update a single record by ID.*

In [ ]:
show_doc(get_by_id)

---

### get_by_id

>      get_by_id (db:fastsql.core.Database, table_name:str, id_value:Any,
>                 id_col:str='id')

*Get a single record by ID.*

---

## 🔧 Utilities

Transaction management, pagination, and batch processing.

| Function | Purpose |
|----------|--------|
| `with_transaction` | Context manager for atomic operations |
| `paginate_sql` | Add LIMIT/OFFSET to queries |
| `batch_execute` | Process large lists in memory-safe chunks |

In [ ]:
#| export

@contextmanager
def with_transaction(db: Database):
    """Context manager for safe transaction handling with auto-rollback on error."""
    try:
        yield db
        db.conn.commit()
    except Exception as e:
        db.conn.rollback()
        raise e

def paginate_sql(sql: str, page: int, page_size: int) -> str:
    """Add LIMIT/OFFSET pagination to a SQL query."""
    offset = (page - 1) * page_size
    return f"{sql.rstrip(';')} LIMIT {page_size} OFFSET {offset}"

def batch_execute(db: Database, operation_func, items: List[Any], batch_size: int = 100) -> None:
    """Execute an operation on items in batches with commits after each batch."""
    for i in range(0, len(items), batch_size):
        batch = items[i:i + batch_size]
        try:
            for item in batch:
                operation_func(db, item)
            db.conn.commit()
        except Exception as e:
            logger.error(f"Batch execution failed at batch {i//batch_size + 1}: {e}", exc_info=True)
            db.conn.rollback()
            raise Exception(f"Batch execution failed at batch {i//batch_size + 1}: {str(e)}") from e

In [ ]:
show_doc(batch_execute)

---

### batch_execute

>      batch_execute (db:fastsql.core.Database, operation_func, items:List[Any],
>                     batch_size:int=100)

*Execute an operation on items in batches with commits after each batch.*

In [ ]:
show_doc(paginate_sql)

---

### paginate_sql

>      paginate_sql (sql:str, page:int, page_size:int)

*Add LIMIT/OFFSET pagination to a SQL query.*

In [ ]:
show_doc(with_transaction)

---

### with_transaction

>      with_transaction (db:fastsql.core.Database)

*Context manager for safe transaction handling with auto-rollback on error.*

---

## 💰 Money Helpers

Convert between dollar amounts and integer cents for database storage.

> ⚠️ **Why cents?** fastsql only supports `int` and `str` types — storing money as cents avoids floating-point precision issues.

| Function | Direction | Example |
|----------|-----------|--------|
| `to_cents` | `"$150.00"` → `15000` | Store in DB |
| `from_cents` | `15000` → `"$150.00"` | Display to user |


In [ ]:
#| export

def to_cents(dollars: str | float | None) -> int | None:
    """Convert dollar amount to integer cents for database storage."""
    if dollars is None or dollars == '':
        return None
    try:
        return int(float(dollars) * 100)
    except (ValueError, TypeError):
        return None


def from_cents(cents: int | None) -> str:
    """Convert integer cents to formatted dollar string for display."""
    if cents is None:
        return "$0.00"
    dollars = cents / 100
    if dollars < 0:
        return f"-${abs(dollars):,.2f}"
    return f"${dollars:,.2f}"

In [ ]:
show_doc(to_cents)

---

### to_cents

>      to_cents (dollars:str|float|None)

*Convert dollar amount to integer cents for database storage.*

In [ ]:
show_doc(from_cents)

---

### from_cents

>      from_cents (cents:int|None)

*Convert integer cents to formatted dollar string for display.*

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()